# 优化模型参数

- 译文：https://pytorch.apachecn.org/2.0/tutorials/beginner/basics/optimization_tutorial
- 原文：https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

## 概览

- 训练模型是一个迭代过程：每个 epoch 包含一次训练循环与一次验证/测试循环。
- 每次训练迭代包括：前向传播、计算损失、反向传播（梯度计算）、根据梯度更新参数。
- 本文展示完整的训练/测试循环实现、超参数设置与示例输出说明。

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# 加载数据集
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# 模型定义（与之前章节一致）
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 超参数

常见超参数示例：迭代次数 (epochs)、批大小 (batch_size)、学习率 (learning_rate)。

下面设置示例超参数：

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# 注意：DataLoader 的 batch_size 与此处变量应保持一致（示例中已在前面设置为64）。

## 损失函数与优化器

- 损失函数衡量模型输出与真实标签之间的差距。分类任务常用 `nn.CrossEntropyLoss()`。
- 优化器根据梯度更新模型参数，常见优化器有 SGD、Adam 等。下面用 SGD 示例：

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss_val, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss_val:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# 运行训练与测试循环（示例设置 epochs）
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

## 小结与建议

- 训练步骤：清零梯度 -> 前向传播 -> 计算损失 -> 反向传播 -> 优化器更新。
- 注意在训练前调用 `model.train()`，在测试/验证时调用 `model.eval()` 并使用 `torch.no_grad()` 来禁用梯度计算。